# Data Analysis Part Andrea

Linking number of edit events since last successful build to probability of build succeeding

In [1]:
import pandas as pd
import numpy as np

# Import Data


In [21]:
editEvents = pd.read_csv("data/editEvents.csv")
buildEvents = pd.read_csv("data/buildEvents.csv")

### Edit Events

In [22]:
print(f'No of edit events: {len(editEvents)}')
editEvents.head()

No of edit events: 497459


,sessionID,timestamp
0,034dd66a-7d1b-43eb-a245-82f5d1cdf6ca,2016-07-08T16:23:50.439936700+02:00
1,034dd66a-7d1b-43eb-a245-82f5d1cdf6ca,2016-07-08T16:24:22.582624100+02:00
2,034dd66a-7d1b-43eb-a245-82f5d1cdf6ca,2016-07-08T16:24:24.591515+02:00
3,8d22b7b4-58c5-4101-b754-d3fa842ba62d,2016-07-08T17:48:12.651904400+02:00
4,8d22b7b4-58c5-4101-b754-d3fa842ba62d,2016-07-08T17:52:09.494520100+02:00


In [18]:
editEvents.describe()

,sessionID,timestamp
count,497459,497459
unique,2874,496546
top,8d0ea603-57cd-4b1f-b3cf-ce39ec9203c7,2016-07-08T10:02:00.260073600+02:00
freq,17006,2


### Build Events

In [15]:
print(f'No of build events: {len(buildEvents)}')
buildEvents.head()

No of test events: 14957


,sessionID,timestamp,buildSuccessful
0,8d22b7b4-58c5-4101-b754-d3fa842ba62d,2016-07-08T18:02:25.521489400+02:00,True
1,8d22b7b4-58c5-4101-b754-d3fa842ba62d,2016-07-08T18:02:47.427946300+02:00,True
2,ac705dcb-0848-4448-847d-70f0f8f0315a,2016-07-08T18:17:57.693892300+02:00,True
3,f23af4c6-66b5-4d9f-b6e0-83de319b70c5,2016-07-08T20:06:38.448145+02:00,True
4,f23af4c6-66b5-4d9f-b6e0-83de319b70c5,2016-07-08T20:08:21.168403+02:00,True


In [19]:
buildEvents.describe()

,sessionID,timestamp,buildSuccessful
count,14957,14956,14956
unique,1529,14933,2
top,26f3dadd-c2e3-414b-8c92-78df1dc356c8,2016-12-16T10:22:34.221873400+01:00,True
freq,154,2,12889


# Process Data

1. Step: For each sessionID, get timestamp with successful build
2. Step: For each passed build, check whether another build follows
    - if yes: count nr of edits until this next build, and record test result of next test
    - if no: dont count edits

In [8]:
def get_all_build(sessionID):
    all_build = buildEvents[buildEvents["sessionID"] == sessionID]
    return all_build.sort(["timestamp"]).values


In [ ]:
def get_successful_build(sessionID):
    all_successful_build = buildEvents[(buildEvents["sessionID"] == sessionID) & (buildEvents["buildSuccessful"] == True)] 
    all_successful_build.sort(["timestamp"])
    return all_successful_build["timestamp"].values


In [9]:
def get_nr_edits_between_build(sessionID,timeLastSuccessfullBuild, timeNextBuild):
    totalEdit = editEvents[editEvents["sessionID"] == sessionID]["timestamp"].sort(["timestamp"])
    testBetweenSuccessBuillAndBuild = totalEdit[(totalEdit["timestamp"] < timeNextBuild) & (totalEdit["timestamp"] > timeLastSuccessfullBuild)]
    return testBetweenSuccessBuillAndBuild.values.size


In [34]:
def edits_from_pass_to_next_test(sessionID):
    result = []

    # get all successful build of session 
    allSuccessfulBuild = get_successful_build(sessionID)
    allBuild = get_all_build(sessionID)
    numOfSuccessfulBuild = allSuccessfulBuild.size
    numOfBuild = np.size(allBuild,0)
    # iterate over each passed build
    for s in range(numOfSuccessfulBuild):
        # if build follows, count edits
        if allBuild[-1][1]>allSuccessfullBuild[s]:
            for b in range(numOfBuild):
                if allBuild[b][1]>allSuccessfulBuild[s]:
                    break
            timeLastSuccessfullBuild=allSuccessfulBuild[s]
            timeNextBuild=allBuild[b][1]
            resultNextBuild=allBuild[b][2]
            nrOfEdit=get_nr_edits_between_build(sessionID,timeLastSuccessfullBuild, timeNextBuild)
            sessionID = allBuild[b][0]
            
            result.append([sessionID, timeLastSuccessfullBuild, timeNextBuild, nrOfEdit, resultNextBuild])
            
    return result

### Iterate over all sessions and create DataFrame

In [ ]:
final_result=[]
allSessionID=buildEvents.sessionID.unique()
for sessionID in allSessionID:
    final_reult.append(edits_from_pass_to_next_test(sessionID))
#create DataFrame
column_labels = ['sessionID','timestampSuccessBuild','timestampNextBuild', '#editsUntilNextBuild', 'nextBuildResult']
df = pd.DataFrame(resultArray, columns=column_labels)
        

## Build a Model

In [ ]:
#TODO: probability of successful build given number of edits after last successful build